In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

#### Handle class imbalance

In [4]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

### Track Experiments

In [5]:
models = [
    ("Logistic Regression", 
    {"solver":'liblinear', "C":1}, 
    LogisticRegression(), 
    (X_train, y_train), 
    (X_test, y_test) 
    ), 
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [6]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]

    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    # print(report)
    reports.append(report)

In [7]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [14]:
# Initialize MLflow
mlflow.set_experiment("AnomalyDetection")
mlflow.set_tracking_uri("http://127.0.0.1:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/02/01 17:16:08 INFO mlflow.tracking.fluent: Experiment with name 'AnomalyDetection' does not exist. Creating a new experiment.
2025/02/01 17:16:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/594492489375008555/runs/d72465a9e3774e8ea7868c4a8522db0a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/594492489375008555


2025/02/01 17:16:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/594492489375008555/runs/1b0b44c4230e4a2c82274e9242241f5e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/594492489375008555


2025/02/01 17:17:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://127.0.0.1:5000/#/experiments/594492489375008555/runs/c10187af29734f4b9214bc112789e886
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/594492489375008555


2025/02/01 17:17:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/594492489375008555/runs/7b289a54942345bcacef13b02af28c44
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/594492489375008555


### Register the Model

In [27]:
model_name = "XGB-Smote" 
run_id = input("Enter Run ID:")
model_uri = f"runs:/{run_id}/model"
result = mlflow.register_model(
    model_uri, model_name
    # "runs:/d16076a3ec534311817565e6527539c0/sklearn-model", "sk-learn-random-forest-reg"
)

Enter Run ID: 7b289a54942345bcacef13b02af28c44


Successfully registered model 'XGB-Smote'.
2025/02/01 17:49:02 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1
Created version '1' of model 'XGB-Smote'.


### Load the model

In [28]:
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

### Transition the Model to Production

In [29]:
dev_model_uri = f"models:/{model_name}@challenger"
prod_model = 'anomaly-detection-prod'

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=dev_model_uri, dst_name=prod_model)

Successfully registered model 'anomaly-detection-prod'.
Copied version '1' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1738412366199, current_stage='None', description='', last_updated_timestamp=1738412366199, name='anomaly-detection-prod', run_id='7b289a54942345bcacef13b02af28c44', run_link='', source='models:/XGB-Smote/1', status='READY', status_message='', tags={}, user_id='', version='1'>

In [30]:
model_uri = f"models:/{prod_model}@champion"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])